In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import os
import json
BASE_PATH = os.environ.get("BASE_PATH", os.path.join(os.sep, 'Projects', 'Python_projects', 'Covid19_Analysis'))
BASE_DATA_PATH = os.environ.get("BASE_DATA_PATH", os.path.join(os.sep, 'Projects', 'Python_projects', 'Covid19_Analysis', 'data'))

# Loading various Datasets

### timeseries

In [ ]:
ts_df = pd.read_csv('https://api.covid19india.org/csv/latest/case_time_series.csv')
ts_df.shape

In [ ]:
total_cases_df = ts_df[['Date', 'Date_YMD', 'Total Confirmed', 'Total Recovered', 'Total Deceased']]
daily_cases_df = ts_df[['Date', 'Date_YMD', 'Daily Confirmed', 'Daily Recovered', 'Daily Deceased']]

In [ ]:
total_cases_df.tail()

In [ ]:
tot = {
    'name': 'Total Confirmed',
    'x': list(total_cases_df['Date']),
    'y': list(total_cases_df['Total Confirmed'])
}
tot

In [ ]:
total_cases_df['Date_YMD'] = pd.to_datetime(total_cases_df['Date_YMD'])

### Statewise

In [2]:
# state_df = pd.read_csv("https://api.covid19india.org/csv/latest/state_wise.csv")
# state_daily_df = pd.read_csv("https://api.covid19india.org/csv/latest/state_wise_daily.csv")
# state_wise_test_df = pd.read_csv("https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv")
st_df = pd.read_csv("https://api.covid19india.org/csv/latest/states.csv")

In [ ]:
state_df.drop('State_Notes', axis=1, inplace=True)

In [ ]:
st_list = list(state_df['State'][1:])

In [ ]:
state_df.head()

In [ ]:
ss = dict(state_df.loc[1, :])
ss
# ss = {column: col_val for column, col_val in ss.items() if column != 'State'}

In [ ]:
{k:vv for k,v in ss.items() for kk,vv in v.items() if k != "State"}

In [ ]:
state_code_mapper = {k:v for k,v in zip(state_df['State_code'], state_df['State'])}
state_code_mapper

In [ ]:
with open(f'{BASE_DATA_PATH}\\default_objects\\state_code_mapper.json', 'w', encoding='utf-8') as fp:
    json.dump(state_code_mapper, fp)

In [ ]:
state_daily_df.columns = [state_code_mapper.get(i,i) for i in state_daily_df.columns]
state_daily_df.drop('State Unassigned', axis=1, inplace=True)

In [ ]:
state_daily_df.head()

In [ ]:
x = st_df[st_df['State'] == 'Maharashtra']
x.fillna(0.0, axis=1, inplace=True)
x.tail()

In [ ]:
import gc
gc.collect()

In [6]:
import gc 
gc.collect()

0

### Districts

In [ ]:
dist_df = pd.read_csv("https://api.covid19india.org/csv/latest/districts.csv")
dist_df.head()

In [ ]:
dist_df[(dist_df['State'] == 'Maharashtra') & (dist_df['District'] == 'Ahmednagar')]

In [ ]:
dd = pd.read_csv("https://api.covid19india.org/csv/latest/district_wise.csv")
dd.head()

In [ ]:
dd[(dd['State'] == 'Maharashtra') & (dd['District'] == 'Ahmednagar')]

## MASTER DATA - beds, ventilators

#### - Most of the data is NAN

In [ ]:
df = pd.read_csv("https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv")

In [ ]:
df.tail()

In [ ]:
bed_cols = [i for i in df.columns if "bed" in i.lower()]
bed_df = df[["Updated On", "State"] + ['People on Ventilator', 'Total Num Ventilators'] + bed_cols]
bed_df.shape

In [ ]:
bed_df[bed_df['State'] == "Delhi"].tail(20)

In [ ]:
[i for i in df.columns if "ventilator" in i.lower()]

In [ ]:
df[df['State'] == 'Mah'].tail(50)

## Vaccine data

In [20]:
vdf = pd.read_csv("http://api.covid19india.org/csv/latest/cowin_vaccine_data_statewise.csv")

In [21]:
vdf = vdf[['Updated On', 'State', 'Total Individuals Registered', 'First Dose Administered', 'Second Dose Administered', 'Male(Individuals Vaccinated)', 'Female(Individuals Vaccinated)',
          'Transgender(Individuals Vaccinated)', 'Total Covaxin Administered', 'Total CoviShield Administered', 'Total Doses Administered']]

In [22]:
vdf.rename(columns={'Male(Individuals Vaccinated)': 'Male', 'Female(Individuals Vaccinated)': 'Female', 'Transgender(Individuals Vaccinated)': 'Transgender'}, inplace=True)

In [ ]:
x = vdf[vdf['State'] == 'Maharashtra'].to_dict()
xx = {k:list(v.values()) for k,v in x.items()}
xx

In [ ]:
temp = dict(ddf[ddf['State'] == 'Maharashtra'].iloc[0])
tp = {
    "state": temp.pop("State"),
    "x": list(temp.keys()),
    "y": list(temp.values())
}
tp

In [ ]:
vdf.head()

In [15]:
for col in vdf.columns[2:]:
    vdf[f'delta_{"_".join(col.split())}'.lower()] = vdf[col].diff()

# Getting all delta values

In [23]:
samp = vdf[vdf['State'] == 'Maharashtra']

In [24]:
for col in samp.columns[2:]:
    samp[f'delta_{"_".join(col.split())}'.lower()] = samp[col].diff()
    
samp.fillna(0.0, axis=1, inplace=True)

c:\projects\virtual_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
samp.head()

,Updated On,State,Total Individuals Registered,First Dose Administered,Second Dose Administered,Male,Female,Transgender,Total Covaxin Administered,Total CoviShield Administered,Total Doses Administered,delta_total_individuals_registered,delta_first_dose_administered,delta_second_dose_administered,delta_male,delta_female,delta_transgender,delta_total_covaxin_administered,delta_total_covishield_administered,delta_total_doses_administered
2205,16/01/2021,Maharashtra,812649.0,5726.0,0.0,3668.0,2057.0,1.0,85.0,5641.0,5726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2206,17/01/2021,Maharashtra,814714.0,6521.0,0.0,3953.0,2566.0,2.0,94.0,6427.0,6521,2065.0,795.0,0.0,285.0,509.0,1.0,9.0,786.0,795.0
2207,18/01/2021,Maharashtra,811257.0,6151.0,0.0,3569.0,2581.0,1.0,105.0,6046.0,6151,-3457.0,-370.0,0.0,-384.0,15.0,-1.0,11.0,-381.0,-370.0
2208,19/01/2021,Maharashtra,850966.0,13699.0,0.0,6328.0,7367.0,4.0,214.0,13485.0,13699,39709.0,7548.0,0.0,2759.0,4786.0,3.0,109.0,7439.0,7548.0
2209,20/01/2021,Maharashtra,870296.0,23880.0,0.0,9658.0,14205.0,17.0,439.0,23441.0,23880,19330.0,10181.0,0.0,3330.0,6838.0,13.0,225.0,9956.0,10181.0


# Statistics of Vaccine administration

In [26]:
samp.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,Total Individuals Registered,First Dose Administered,Second Dose Administered,Male,Female,Transgender,Total Covaxin Administered,Total CoviShield Administered,Total Doses Administered,delta_total_individuals_registered,delta_first_dose_administered,delta_second_dose_administered,delta_male,delta_female,delta_transgender,delta_total_covaxin_administered,delta_total_covishield_administered,delta_total_doses_administered
count,105.00000,105.00000,105.00000,105.00000,105.00000,105.00000,105.00000,105.00000,105.00000,104.00000,104.00000,104.00000,104.00000,104.00000,104.00000,104.00000,104.00000,104.00000
mean,4417032.32381,3791049.41905,542957.93333,2035530.22857,1755041.64762,477.54286,417064.84762,3745593.84762,4334007.35238,122584.50000,130343.37500,25215.48077,70100.82692,60226.04808,16.50000,17415.91346,138142.94231,155558.85577
std,3972174.70668,4316323.23989,649605.02201,2333644.13661,1982475.71789,506.26758,566891.54366,4413923.49748,4943331.09022,139412.60046,200218.53856,36341.25369,105091.56698,95228.92998,33.13784,27960.47258,255959.36584,215145.30979
min,811257.00000,5726.00000,0.00000,3569.00000,2057.00000,1.00000,85.00000,5641.00000,5726.00000,-35751.00000,-370.00000,0.00000,-384.00000,0.00000,-1.00000,0.00000,-381.00000,-370.00000
25%,1443232.00000,579241.00000,0.00000,255793.00000,323314.00000,134.00000,5018.00000,574223.00000,579241.00000,21634.75000,30527.25000,0.00000,14453.00000,10253.50000,2.00000,265.25000,29728.00000,33813.50000
50%,2346229.00000,1536355.00000,305927.00000,839907.00000,696211.00000,237.00000,29698.00000,1506657.00000,1842282.00000,73178.00000,51323.00000,15462.00000,29855.50000,27056.00000,9.00000,2444.50000,49455.50000,68285.00000
75%,6841397.00000,6841397.00000,805729.00000,3682530.00000,3158056.00000,811.00000,876863.00000,6770263.00000,7647126.00000,191237.75000,190121.25000,28306.75000,102507.50000,85473.25000,21.00000,27872.00000,190660.50000,226283.75000
max,13561437.00000,13561437.00000,2622410.00000,7294154.00000,6265566.00000,1717.00000,1811340.00000,14372507.00000,16183847.00000,916850.00000,1795839.00000,199597.00000,934006.00000,861518.00000,315.00000,206919.00000,2394649.00000,1817397.00000


In [28]:
ss = samp.describe().apply(lambda s: s.apply('{0:.5f}'.format))

In [29]:
ss.to_dict()

{'Total Individuals Registered': {'count': '105.00000',
  'mean': '4417032.32381',
  'std': '3972174.70668',
  'min': '811257.00000',
  '25%': '1443232.00000',
  '50%': '2346229.00000',
  '75%': '6841397.00000',
  'max': '13561437.00000'},
 'First Dose Administered': {'count': '105.00000',
  'mean': '3791049.41905',
  'std': '4316323.23989',
  'min': '5726.00000',
  '25%': '579241.00000',
  '50%': '1536355.00000',
  '75%': '6841397.00000',
  'max': '13561437.00000'},
 'Second Dose Administered': {'count': '105.00000',
  'mean': '542957.93333',
  'std': '649605.02201',
  'min': '0.00000',
  '25%': '0.00000',
  '50%': '305927.00000',
  '75%': '805729.00000',
  'max': '2622410.00000'},
 'Male': {'count': '105.00000',
  'mean': '2035530.22857',
  'std': '2333644.13661',
  'min': '3569.00000',
  '25%': '255793.00000',
  '50%': '839907.00000',
  '75%': '3682530.00000',
  'max': '7294154.00000'},
 'Female': {'count': '105.00000',
  'mean': '1755041.64762',
  'std': '1982475.71789',
  'min': '

In [30]:
df = pd.read_csv('http://api.covid19india.org/csv/latest/vaccine_doses_statewise.csv')

In [31]:
df.head()

,State,16/01/2021,18/01/2021,19/01/2021,20/01/2021,21/01/2021,22/01/2021,23/01/2021,24/01/2021,25/01/2021,26/01/2021,27/01/2021,28/01/2021,29/01/2021,30/01/2021,31/01/2021,01/02/2021,02/02/2021,03/02/2021,04/02/2021,05/02/2021,06/02/2021,07/02/2021,08/02/2021,09/02/2021,10/02/2021,11/02/2021,12/02/2021,13/02/2021,14/02/2021,15/02/2021,16/02/2021,17/02/2021,18/02/2021,19/02/2021,20/02/2021,21/02/2021,22/02/2021,23/02/2021,24/02/2021,25/02/2021,26/02/2021,27/02/2021,28/02/2021,01/03/2021,02/03/2021,03/03/2021,04/03/2021,05/03/2021,06/03/2021,07/03/2021,08/03/2021,09/03/2021,10/03/2021,11/03/2021,12/03/2021,13/03/2021,14/03/2021,15/03/2021,16/03/2021,17/03/2021,18/03/2021,19/03/2021,20/03/2021,21/03/2021,22/03/2021,23/03/2021,24/03/2021,25/03/2021,26/03/2021,27/03/2021,28/03/2021,29/03/2021,30/03/2021,31/03/2021,01/04/2021,02/04/2021,03/04/2021,04/04/2021,05/04/2021,06/04/2021,07/04/2021,08/04/2021,09/04/2021,10/04/2021,11/04/2021,12/04/2021,13/04/2021,14/04/2021,15/04/2021,16/04/2021,17/04/2021,18/04/2021,19/04/2021,20/04/2021,21/04/2021,22/04/2021,23/04/2021,24/04/2021,25/04/2021,26/04/2021,27/04/2021,28/04/2021,29/04/2021
0,Andaman and Nicobar Islands,225,442,644,644,1032,1466,1998,2019,2369,2369,2385,2656,2727,2727,2727,2727,2727,2772,2938,3161,3161,3397,3397,3413,3413,3454,3454,3646,3646,4029,4029,4227,4842,5348,6152,6152,7312,7583,7762,8419,8556,8556,8556,9137,9501,9501,9501,11119,11842,12052,12888,13470,13797,14288,14874,15203,15203,NaN,16194,16194,17071,17506,18132,18132,18990,19752,20002,20672,20991,21361,21361,21361,22088,22088,22874,22867,23432,24076,25217,26850,28269,30799,33217,35447,37671,42285,48901,55598,60898,67734,74108,78236,83085,86926,89072,91977,94166,96109,97392,99109,100588,101770,102998
1,Andhra Pradesh,18412,46680,58495,91778,115365,133298,147030,147030,156120,156129,163727,171683,179038,187252,187252,187252,187252,215171,243243,272190,299649,299649,314316,325538,335268,343813,351993,362341,362341,376444,390665,411783,438766,469815,493471,503323,529820,556810,592903,634449,668944,668944,668944,683633,699364,722601,748938,785878,834025,834025,882244,930160,973239,981721,1031926,1077813,1077813,NaN,1235282,1316192,1406055,1503787,1611084,1611084,1741878,1850350,1972034,2113981,2219053,2341955,2350279,2408323,2504479,2605169,2735502,2907602,3007468,3053542,3179087,3266611,3389990,3497015,3571233,3806708,3893003,3913719,3918529,4563073,4597577,4603804,4607975,4613024,4756984,4927736,4968504,5330080,5555014,5670368,5678628,5986822,6177974,6282487,6354053
2,Arunachal Pradesh,829,2805,2805,3023,4682,5956,6511,6511,7307,7307,7307,8656,9265,9651,9651,9791,9791,9846,10889,11834,12346,12346,13479,13480,13480,14322,15098,16094,16094,18187,18187,18187,21300,23559,23743,23743,26650,27916,27930,30524,32120,32120,32120,34881,37031,37162,40830,43918,45667,45667,48474,50384,50384,53386,55709,57056,57056,NaN,62168,62168,65786,68198,69919,69919,73426,75474,75593,78893,81533,83450,83450,83808,85889,85889,93159,100226,104436,106266,107592,112266,114031,120397,126477,131304,135099,144244,151893,158211,161283,166715,170942,172669,180287,186071,190500,197578,204542,209180,210345,216946,222159,227858,234727
3,Assam,3528,5542,7418,7585,10676,13881,13881,13881,19837,19837,19945,28918,36932,38106,38106,39724,42435,43607,60556,77225,87269,88585,99889,108887,110977,117607,125038,129781,129873,132475,135419,137224,146748,157532,164309,165804,179102,190374,195420,211037,223581,223581,223581,230799,251303,270406,316106,357619,394401,405394,441699,469387,488139,508981,528177,538913,539254,NaN,580695,590838,609827,627818,645191,647269,678659,810619,837127,926350,986804,1004038,1004248,1006703,1039302,1048974,1083743,1167124,1206058,1220939,1276382,1320944,1344908,1405680,1454577,1486294,1498729,1531037,1550113,1553903,1555461,1564490,1586438,1597535,1677769,1771560,1825312,1900839,1961751,2006178,2020742,2105788,2193900,2244119,2351160
4,Bihar,18169,33389,42085,47433,63541,63620,76125,76125,88450,88450,89074,107174,110396,146015,148293,184

In [41]:
samp.columns[2:]

Index(['Total Individuals Registered', 'First Dose Administered',
       'Second Dose Administered', 'Male', 'Female', 'Transgender',
       'Total Covaxin Administered', 'Total CoviShield Administered',
       'Total Doses Administered', 'delta_total_individuals_registered',
       'delta_first_dose_administered', 'delta_second_dose_administered',
       'delta_male', 'delta_female', 'delta_transgender',
       'delta_total_covaxin_administered',
       'delta_total_covishield_administered',
       'delta_total_doses_administered'],
      dtype='object')

In [45]:
tot = vdf[vdf['State'] == 'India']

In [48]:
vdf.head()

,Updated On,State,Total Individuals Registered,First Dose Administered,Second Dose Administered,Male,Female,Transgender,Total Covaxin Administered,Total CoviShield Administered,Total Doses Administered
0,16/01/2021,India,6943622.0,48276.0,0.0,23757.0,24517.0,2.0,579.0,47697.0,48276
1,17/01/2021,India,7119567.0,58604.0,0.0,27348.0,31252.0,4.0,635.0,57969.0,58604
2,18/01/2021,India,7150780.0,99449.0,0.0,41361.0,58083.0,5.0,1299.0,98150.0,99449
3,19/01/2021,India,7423057.0,195525.0,0.0,81901.0,113613.0,11.0,3017.0,192508.0,195525
4,20/01/2021,India,7555207.0,251280.0,0.0,98111.0,153145.0,24.0,3946.0,247334.0,251280


In [49]:
tot_dd = dict(vdf[vdf['State'] == "India"].iloc[0])

In [50]:
tot_dd

{'Updated On': '16/01/2021',
 'State': 'India',
 'Total Individuals Registered': 6943622.0,
 'First Dose Administered': 48276.0,
 'Second Dose Administered': 0.0,
 'Male': 23757.0,
 'Female': 24517.0,
 'Transgender': 2.0,
 'Total Covaxin Administered': 579.0,
 'Total CoviShield Administered': 47697.0,
 'Total Doses Administered': 48276}